# RAG 和 llama-index,Ollama 综合项目


In [1]:
import chromadb
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    get_response_synthesizer,
    Settings,
)
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

In [2]:
# 设置嵌入模型和语言模型
Settings.embed_model = OllamaEmbedding(model_name="yxl/m3e:latest")
Settings.llm = Ollama(model="qwen2.5:0.5b", request_timeout=360)

In [3]:
# 初始化 Chroma 客户端，指定数据存储路径为当前目录下的 chroma_db 文件夹
db = chromadb.PersistentClient(path="./chroma_db")

# 获取或创建名为 "quickstart" 的集合，如果该集合不存在，则创建它
chroma_collection = db.get_or_create_collection("quickstart")

# 使用上述集合创建一个 ChromaVectorStore 实例，以便 llama_index 可以与 Chroma 集合进行交互
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# 创建一个存储上下文，指定向量存储为刚刚创建的 ChromaVectorStore 实例
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [4]:
# 读取文档
documents = SimpleDirectoryReader("C:/Users/Admin/Desktop/Data/").load_data()

# 构建索引
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    transformations=[SentenceSplitter(chunk_size=256)],
)

In [5]:
# 配置检索器
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,  # 返回最相似的前 n 个文档片段
)

# 配置响应合成器
response_synthesizer = get_response_synthesizer()

# 组装查询引擎
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

In [6]:
# 执行查询
response = query_engine.query("数字经济")
print(response)  # 输出查询结果

数字经济是指以数字技术和信息为基础的现代经济形态。它通过将物理世界的信息、过程和服务转换为数字形式来改变传统的生产方式、商业模式和消费行为，展现出其广泛应用和发展潜力。


In [7]:
chat_engine = index.as_chat_engine()
respon = chat_engine.chat("数字经济是什么")
respon.response.strip()

'数字经济，简称"数字经济",是以数据为关键生产要素,以互联网技术为重要载体,促进新旧动能转换和优化升级的经济形态。'

In [13]:
def answer_question(message, history):

    # 构建会话历史
    prompt = "结合mks文档回答问题"
    if history is not None:
        for h in history:
            prompt += f"用户：{h[0]}\n助手：{h[1]}\n"
    prompt += f"用户：{message}\n助手："

    # 使用聊天引擎生成回答
    response = chat_engine.chat(prompt)

    # 更新历史记录
    history = history or []
    history.append((message, response.response.strip()))

    return "", history


def submit_file():
    pass

In [9]:
import gradio as gr


# 模拟用户验证函数
def verify_user(username, password):
    # 这里可以替换为实际的用户验证逻辑
    if username == "1" and password == "1":
        return True
    return False


# 登录函数
def login(username, password):
    if verify_user(username, password):
        return gr.update(visible=False), gr.update(visible=True), "登陆成功!"
    else:
        return gr.update(visible=True), gr.update(visible=False), "用户名或密码错误!"


# 模拟专家验证函数
def verify_expert(username, password):
    # 这里可以替换为实际的用户验证逻辑
    if username == "2" and password == "2":
        return True
    return False


# 专家登录函数
def expert_login(username, password):
    if verify_expert(username, password):
        return gr.update(visible=False), gr.update(visible=True), "登陆成功!"
    else:
        return gr.update(visible=True), gr.update(visible=False), "用户名或密码错误!"


# 退出登录函数
def logout():
    return gr.update(visible=True), gr.update(visible=False), ""

# 前端设计


In [14]:
with gr.Blocks() as demo:
    # 登录界面
    with gr.Column(visible=True) as login_interface:
        gr.Markdown("<h1 style='text-align: center;'>登录</h1>")
        username = gr.Textbox(label="用户名")
        password = gr.Textbox(label="密码", type="password")
        login_button = gr.Button("登录", scale=1)
        login_message = gr.Markdown("")
        expert_login_button = gr.Button("专家登录", scale=1)

    # 聊天界面
    with gr.Column(visible=False) as chat_interface:
        gr.Markdown("<h1 style='text-align: center;'>AI 问答界面</h1>")
        with gr.Row():
            gr.Markdown("")
            logout_button = gr.Button("退出登录", elem_id="logout_button")
        chatbot = gr.Chatbot()
        msg = gr.Textbox(show_label=False, placeholder="请输入您的消息...")
        submit = gr.Button("提交对话")
        clear = gr.Button("清除对话")

        msg.submit(answer_question, [msg, chatbot], [msg, chatbot])
        submit.click(answer_question, [msg, chatbot], [msg, chatbot])
        clear.click(lambda: None, None, chatbot, queue=False)
        logout_button.click(
            logout, outputs=[login_interface, chat_interface, login_message]
        )
    # 专家聊天界面
    with gr.Column(visible=False) as expert_chat_interface:
        gr.Markdown("<h1 style='text-align: center;'>专家引导界面</h1>")
        with gr.Row():
            gr.Markdown("")
            logout_button = gr.Button("退出登录", elem_id="logout_button")
        chatbot = gr.Chatbot()
        with gr.Row():
            with gr.Column():
                msg = gr.Textbox(show_label=False, placeholder="请输入您的消息...")
                submit = gr.Button("提交对话")
                clear = gr.Button("清除对话")
            with gr.Column():
                file = gr.File(label="上传所需文件")
                submit_file = gr.Button("提交文件")
                clear_file = gr.Button("清除文件")

        msg.submit(answer_question, [msg, chatbot], [msg, chatbot])
        submit.click(answer_question, [msg, chatbot], [msg, chatbot])
        clear.click(lambda: None, None, chatbot, queue=False)
        logout_button.click(
            logout, outputs=[login_interface, expert_chat_interface, login_message]
        )

    # 绑定登录按钮的点击事件
    login_button.click(
        login,
        inputs=[username, password],
        outputs=[login_interface, chat_interface, login_message],
    )
    # 绑定专家登陆事件
    expert_login_button.click(
        expert_login,
        inputs=[username, password],
        outputs=[login_interface, expert_chat_interface, login_message],
    )

demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
